### **Import Reuqired Libraries**

In [137]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder,StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,classification_report
import tensorflow as tf
from scikeras.wrappers import KerasClassifier

### **Import Datasets**

In [138]:
train_data=pd.read_csv("../dataset/train_data.csv")
test_data=pd.read_csv("../dataset/test_data.csv")

In [139]:
print(f"Shape of the Training data:{train_data.shape}")
print(f"Shape of the Testing data:{test_data.shape}")

Shape of the Training data:(1550, 13)
Shape of the Testing data:(388, 13)


### **Drop Unecessary Columns**

In [140]:
train_data=train_data=train_data.drop(["Unnamed: 0"],axis=1)
test_data=test_data.drop(["Unnamed: 0"],axis=1)

In [141]:
##split train data into the X_train and Y_train
X_train=train_data.drop(["delivery_status"],axis=1)
Y_train=train_data["delivery_status"]

In [142]:
##Adjust the datatype of  assembly_service_requested column
X_train["assembly_service_requested"]=X_train["assembly_service_requested"].astype("object")

### **Model Building**

In [143]:
# ### Numbercal columns 
numerical_features=X_train.select_dtypes(include="number").columns.to_list()
categorical_features=X_train.select_dtypes(include="object").columns.to_list()

###Nominal variables
categorical_features.remove("brand")
##Ordinal variables
ordinal_features=["brand"]


In [144]:
### Define pipelines
nominal_pipeline=Pipeline(steps=[
    ("One-Hot-Encoder",OneHotEncoder(sparse_output=False,handle_unknown="ignore"))
])
numerical_pipeline=Pipeline(steps=[
    ("Standard Scaler",StandardScaler())
])

ordinal_pipeline=Pipeline(steps=[
    ("Ordinal-Encoder",OrdinalEncoder())
])


transfomer=ColumnTransformer(transformers=[
    ("Numerical Pipeline",numerical_pipeline,numerical_features),
    ("Nominal Pipeline",nominal_pipeline,categorical_features),
    ("Ordinal Pipeline",ordinal_pipeline,ordinal_features)
])

final_pipeline=Pipeline(steps=[
    ("Transfomer",transfomer)
  
])

In [145]:
X_train_preprocessed = final_pipeline.named_steps["Transfomer"].fit_transform(X_train)

# Get the ColumnTransformer from the pipeline
preprocessor = final_pipeline.named_steps["Transfomer"]

# Get feature names from the ColumnTransformer
feature_names = preprocessor.get_feature_names_out()
X_train_preprocessed=pd.DataFrame(X_train_preprocessed,columns=feature_names)

In [146]:
##split train data into the X_train and Y_train
X_test=test_data.drop(["delivery_status"],axis=1)
Y_test=test_data["delivery_status"]

##Adjust the datatype of  assembly_service_requested column
X_test["assembly_service_requested"]=X_test["assembly_service_requested"].astype("object")

X_test_preprocessed = final_pipeline.named_steps["Transfomer"].transform(X_test)
X_test_preprocessed=pd.DataFrame(X_test_preprocessed,columns=feature_names)

In [147]:
##mapping y variable with the numeric values
y_map={"Failed Delivery":0,"On Going":1,"Delivered":2}
Y_train=Y_train.map(y_map)
Y_test=Y_test.map(y_map)

In [148]:
##implementing ANN using tensorflow
model=tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_preprocessed.shape[1],)), ##Input Layer
    tf.keras.layers.Dense(10,activation="relu"), ## First Hidden Layer
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(6,activation="relu"), ##Second Hidden Layer
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(8,activation="relu"), ##Third Hidden Layer
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(3,activation="softmax") ##Outpur Layer
])

model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model.fit(X_train_preprocessed,Y_train,validation_data=(X_test_preprocessed,Y_test),epochs=200,batch_size=20,verbose=1)

Epoch 1/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3974 - loss: 1.3910 - val_accuracy: 0.4124 - val_loss: 1.1087
Epoch 2/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4303 - loss: 1.2750 - val_accuracy: 0.4485 - val_loss: 1.0704
Epoch 3/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4310 - loss: 1.2134 - val_accuracy: 0.4613 - val_loss: 1.0508
Epoch 4/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4355 - loss: 1.1555 - val_accuracy: 0.4691 - val_loss: 1.0398
Epoch 5/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4510 - loss: 1.1099 - val_accuracy: 0.4871 - val_loss: 1.0320
Epoch 6/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4613 - loss: 1.0821 - val_accuracy: 0.4897 - val_loss: 1.0286
Epoch 7/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4523 - loss: 1.0747 - val_accuracy: 0.4923 - val_loss: 1.0264
Epoch 8/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4594 - loss: 1.0639 - val_accuracy: 0.4974 - v

### **Model Evaluation**

In [149]:
y_pred=model.predict(X_test_preprocessed)
y_pred

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


array([[0.5029654 , 0.31712088, 0.1799137 ],
       [0.50303733, 0.32243487, 0.17452785],
       [0.5090768 , 0.31156293, 0.17936027],
       ...,
       [0.5097751 , 0.31298563, 0.17723933],
       [0.5092028 , 0.3122407 , 0.17855649],
       [0.5096464 , 0.31309786, 0.17725569]],
      shape=(388, 3), dtype=float32)

In [150]:
y_pred=np.argmax(y_pred,axis=1)

In [ ]:
accuracy=accuracy_score(Y_test,y_pred)

c:\Users\ASUS\anaconda3\envs\pyfix\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ASUS\anaconda3\envs\pyfix\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ASUS\anaconda3\envs\pyfix\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0]

In [ ]:
print(f"Accuracy of the ANN model:{accuracy}")

Accuracy of the ANN model:0.49226804123711343
Classification Report:
